In [17]:
import time 
import selenium 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import os
import PIL
from PIL import Image
#from io import bytesIO
import io
import base64 

class Data():    

    
    def __init__(self,search_name):
        DRIVER_PATH = '/home/trey/Sharpest-Minds-Project/chromedriver'
        self.wd = webdriver.Chrome(DRIVER_PATH)
        search_url = "https://www.google.com/search?q={q}&sxsrf=ALeKk02zAb9RaNNb-qSenTEJh1i2XX480w:1613489053802&source=lnms&tbm=isch&sa=X&ved=2ahUKEwjChJqP2-7uAhVyTTABHdX0CPoQ_AUoAXoECAcQAw&biw=767&bih=841"
        self.wd.get(search_url.format(q=search_name))


#class Save():
    
    def scrape_and_save(self):
        body = self.wd.find_element_by_tag_name("body")
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(2.5)
        main = self.wd.find_elements_by_class_name("rg_i.Q4LuWd")
        links = [main[i].get_attribute('src') for i in range(len(main))]
        self.wd.quit()
        images = []
        for image in links:
            # making sure type of data is an image containing the string google encodes for an image
            if type(image) == str:
                # checking conditional text so Image library can open based on based64 conversion
                if image[0:4] == 'data':
                    print("HEre")
                    # replace text so we can open it
                    new = image.replace("data:image/jpeg;base64,","")
                    # adding equals at the end for decoding
                    if new[-2:] != '==':
                        new_edit = new + '=='
                        # image becomes Image object
                        new_image = (Image.open(io.BytesIO(base64.b64decode(new_edit)))).resize((150,150))
                        images.append(new_image)
                    else:
                        new_image = (Image.open(io.BytesIO(base64.b64decode(new)))).resize((150,150))
                        images.append(new_image)
                if image[0:4] == 'http':
                    new = requests.get(image)
                    new_image = Image.open(io.BytesIO(new.content))
                    images.append(new_image)
        # creating directories, saving images, and creating names for images as jpeg files
        #os.mkdir('Images')
        #os.mkdir('Images/training')
        #os.mkdir('Images/testing')
        index = 0 
        for i in images:
            #  images = original_lst = list(range(10))
            # i = images[:int(len(images*0.8))]
            #new_lst = original_lst[:int(len(original_lst)*0.9)]
            #print(new_lst)
            i.save('Images/training'+'/'+ str(index)+'.jpeg')
            index += 1 


In [15]:
data = Data('dogs')

In [16]:
Data.scrape_and_save(data)

HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre
HEre


AttributeError: 'str' object has no attribute 'content'

In [52]:
class Save():
    
    def __init__(links):
        images = []
        for image in links:
            # making sure type of data is an image containing the string google encodes for an image
            if type(image) == str:
                # checking conditional text so Image library can open based on based64 conversion
                if image[0:4] == 'data':
                    print("HEre")
                    # replace text so we can open it
                    new = image.replace("data:image/jpeg;base64,","")
                    # adding equals at the end for decoding
                    if new[-2:] != '==':
                        new_edit = new + '=='
                        # image becomes Image object
                        new_image = (Image.open(BytesIO(base64.b64decode(new_edit)))).resize((150,150))
                        images.append(new_image)
                    else:
                        new_image = (Image.open(BytesIO(base64.b64decode(new)))).resize((150,150))
                        images.append(new_image)
                if image[0:4] == 'http':
                    new = requests.get(image)
                    new_image = Image.open(BytesIO(image.content))
                    images.append(new_image)
        # creating directories, saving images, and creating names for images as jpeg files
        os.mkdir('Images')
        os.mkdir('Images/training')
        os.mkdir('Images/testing')
        index = 0 
        for i in images:
            #  images = original_lst = list(range(10))
            # i = images[:int(len(images*0.8))]
            #new_lst = original_lst[:int(len(original_lst)*0.9)]
            #print(new_lst)
            i.save('Images/training'+'/'+string(index)+'.jpeg')
            index += 1 
        

In [ ]:
# two solutions to propose: 
#    1. break functionality into two seperate classes:
            # -one will take in the result from the previous class and return the data which will be saved in folders
            # - two will keep the function in the first class and figure out how to run 

In [55]:
data = Data('dogs')

In [61]:
Data.scrape_and_save(data)

TypeError: 'Data' object is not iterable

In [15]:
search_name = 'dogs'

In [13]:
scape_and_save('dogs')

NameError: name 'scape_and_save' is not defined